In [1]:
!pip3 install bert-extractive-summarizer
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 23

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
import datasets
import numpy as np
import pandas as pd
from datasets import load_dataset
from datasets import DatasetDict
#from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoConfig
#from transformers import MT5ForConditionalGeneration, T5Tokenizer,AutoConfig
import torch
import torch.nn as nn
from transformers.optimization import Adafactor
import transformers
import torch
from summarizer import Summarizer

Loading Xsum dataset

In [ ]:
raw_datasets = load_dataset("xsum")
train_data = raw_datasets['train']
test_data = raw_datasets['test']

In [ ]:
summarizer = Summarizer()

def extract_important_sentence(example, model):
  text = example['document']
  result = model(text, num_sentences=1)
  return {'important_sentences':result}

In [ ]:
raw_datasets = DatasetDict(
    {
        "train": train_data.select(range(500)),
        "test": test_data.select(range(50)),
    }
)

In [ ]:
raw_datasets =  raw_datasets.map(lambda example:extract_important_sentence(example, summarizer))

In [ ]:
raw_datasets

Loading T5-small model 

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
tokenizer.add_special_tokens({'pad_token':'[PAD]'})

Obtaining keywords from source document

In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
import spacy

# Load the spaCy model

nlp = spacy.load("en_core_web_sm")
# Define a function to extract the top 5 keywords from a text
# Define a function to extract the top 5 keywords from a text
def extract_keywords(text,nlp):
    doc = nlp(text)
    # Get the noun chunks and their frequencies
    noun_chunks = {}
    for chunk in doc.noun_chunks:
        chunk_text = " ".join([token.text for token in chunk if token.text.lower() != "the"])
        if chunk_text.lower() not in nlp.Defaults.stop_words:
            if chunk_text not in noun_chunks:
                noun_chunks[chunk_text] = 1
            else:
                noun_chunks[chunk_text] += 1
    # Sort the noun chunks by frequency and return the top 5
    sorted_chunks = sorted(noun_chunks.items(), key=lambda x: x[1], reverse=True)
    return ', '.join([chunk[0] for chunk in sorted_chunks[:20]])

def keywords(example,nlp):
  # Loop through each sample in the train data and extract the top 5 keywords
    text = example["document"]
    keywords = extract_keywords(text,nlp)
  
    return {"Keywords":keywords}


In [ ]:
nlp = spacy.load("en_core_web_sm")
raw_datasets =  raw_datasets.map(lambda example:keywords(example,nlp))

In [ ]:
tokenizer.add_special_tokens({'pad_token':'[PAD]'})
PAD_IDX = tokenizer.pad_token_id
PAD_IDX

Preprocessing xsum dataset and obtaining final input_ids,attention_mask and label

In [ ]:
prefix_len = 100
def preprocess_function(examples, prefix_len=100, padding=True):
    text_column = 'document'
    summary_column = 'summary'
    important_column = 'important_sentences'
    keyword_column = 'Keywords'
    max_source_length = 512  
    max_target_length = 512

    text = examples[text_column]
    targets = examples[summary_column]
    important = examples[important_column]
    keyword = examples[keyword_column]

    #inputs = " ".join(text.split(' '))[:300]  #for each sample we are only taking 200 words
    #inputs = '<|endoftext|>' + inputs + 'TL;DR:' + targets + '<|endoftext|>'
    #tokenize inputs, label and important sentences
    #model_inputs = tokenizer(inputs, return_tensors="pt", padding=padding, max_length=max_source_length, truncation=True)
    labels = tokenizer(targets, padding=padding, max_length=max_target_length, return_tensors="pt", truncation=True)
    importance_sentences = tokenizer(important, padding="max_length", max_length=prefix_len, return_tensors="pt", truncation=True)
    importance_keywords = tokenizer(keyword, padding="max_length", max_length=prefix_len, return_tensors="pt", truncation=True)
    input_length = importance_sentences['input_ids'].shape[1]
    target_length = labels["input_ids"].shape[1]

    #print(max_source_length,target_length,prefix_len,input_length)
    #concat input ids with importance sentences in the front and label in the back
    importance_sentences['input_ids'] = torch.cat([importance_keywords['input_ids'],importance_sentences['input_ids'],torch.full((1,max_source_length-prefix_len-input_length), PAD_IDX)],1)
    #print('Input shape',model_inputs['input_ids'].shape)
    #concat label with -100 in the front and padding
    labels["input_ids"] = torch.cat([torch.full((1,prefix_len),-100), labels['input_ids'],torch.full((1,max_target_length-prefix_len-target_length), PAD_IDX)],1)
    importance_sentences["labels"] = labels["input_ids"]
    #print(labels['input_ids'].shape)
    #concat attention with 0 in the back
    attention_length = importance_sentences["attention_mask"].shape[1]
    importance_sentences["attention_mask"] = torch.cat([importance_keywords['attention_mask'],importance_sentences['attention_mask'],torch.full((1,max_source_length-prefix_len-attention_length), 1)],1)

In [ ]:
column_names = ['document', 'summary','id','important_sentences','keywords']
new_train_dataset = raw_datasets["train"].map(lambda example : preprocess_function(example,prefix_len),remove_columns=column_names)
new_test_dataset = raw_datasets["test"].map(lambda example : preprocess_function(example,prefix_len),remove_columns=column_names)

In [ ]:
lambda example:extract_important_sentence(example, summarizer)

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

Making batch of xsum dataset

In [ ]:
BATCH_SIZE = 8
new_train_dataset.set_format("torch")
loader = DataLoader(new_train_dataset, batch_size=BATCH_SIZE)

In [ ]:
for batch in loader:
  break

In [ ]:
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                                  n_tokens, 
                                                                                  random_range, 
                                                                                  initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [ ]:
model_setup_for_prompt_tuning=False
if not model_setup_for_prompt_tuning:
  #model.train()

  # Freeze model
  for param in model.parameters():
      param.requires_grad = False

  old_wte = model.get_input_embeddings()

  # Add softembedding module
  s_wte = SoftEmbedding(old_wte,
                        n_tokens=prefix_len,
                        initialize_from_vocab=True).to("cuda")
  model.set_input_embeddings(s_wte)
  if torch.cuda.is_available():
    model = model.cuda()
  # Set up optimizer
  parameters = list(model.parameters())
  # params = [model.transformer.wte.learned_embedding]
  # optimizer = Adafactor(params=params)
  

In [ ]:
optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

In [ ]:
import pickle

iterations = 100
name = "100_1_prefix_with_top_3_imp_sentences_t5"
losses = []
best_loss = float('inf')
for batch in loader:
  for i in range(iterations):
      optimizer.zero_grad()
      input=batch["input_ids"].to("cuda")
      input = input.squeeze(1)
      attention = batch["attention_mask"].to("cuda")
      attention = attention.squeeze(1)
      label = batch["labels"].cuda()
      label = label.squeeze(1)
      output = model(input_ids=input,attention_mask = attention, labels=label)      
      loss = output.loss
      losses.append(loss)
      loss.backward()
      optimizer.step()
      if i%10 == 0:
        print(f"{i}: Loss: {loss}")
  if loss < best_loss:
    best_loss = loss
    #torch.save(model.transformer.wte.learned_embedding, f'{name}.pt')
    pickle.dump(model, open(f"{name}.pkl","wb"))

In [ ]:
final_losses = [loss.item() for loss in losses]